In [11]:
# 匯入模組
import numpy as np
import pandas as pd
from math import sqrt
from scipy.stats import norm
import time
import random
from scipy.stats import skewnorm

In [12]:
# 系統風險因子的因子負荷量
ak1 = 0.8
bk1 = sqrt(1-(ak1)**2)
ak2 = 0.5
bk2 = sqrt(1-(ak2)**2)

# 債務人違約時產生的損失
ck1 = 2
ck2 = 1

# 門檻 和 債務人個數
cc1 = 200
cc2 = 800
# 債務人個數
m = cc1 + cc2

# 個別債務人的違約機率
pk1 = 0.01
pk2 = 0.05

k11 = np.empty(cc1)
k22 = np.empty(cc2)

k11.fill(1)
k22.fill(1)

# SN 模型參數轉換
lk1 = ak1/sqrt(2-(ak1)**2)
lk2 = ak2/sqrt(2-(ak2)**2)

# CSN分配在 1-p 之值，對應回去的Z值
xk1 = skewnorm.ppf(1-pk1, a = lk1, loc = 0, scale = 1)
xk2 = skewnorm.ppf(1-pk2, a = lk2, loc = 0, scale = 1)


#xk1 = skewnorm.ppf(1-p1, 1) #p跟 xk 成反比
#xk2 = skewnorm.ppf(1-p2, 1)
xk1 = xk1*(k11)
xk2 = xk2*(k22)

# 模擬次數
N = 100

In [13]:
xk1,xk2

(array([2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56673879, 2.56673879,
        2.56673879, 2.56673879, 2.56673879, 2.56

In [14]:
LX=[]
LY=[]
for x in range(cc1+1):
    for y in range(cc2+1):
        if(2*x+y > -1):
            #print(x,y)
            LX.append(x)
            LY.append(y)
Ans=pd.concat((pd.DataFrame(LX),pd.DataFrame(LY)),axis=1)
Ans=pd.concat((pd.DataFrame(LX),pd.DataFrame(LY)),axis=1)
Ans.columns=['kx','ky']
# temp = Ans['kx'].drop_duplicates()
# a = temp.index
# Ans = pd.DataFrame(Ans.iloc[a])
# Ans.reset_index(drop=True,inplace=True)
len(Ans)

161001

In [15]:
pY = np.zeros((len(Ans), N))
start_total = time.time()
for i in range(N):
    # 生成(k1, k2)兩群要計算的機率空格
    pk1 = np.zeros(cc1+1)
    pk2 = np.zeros(cc2+1)
    
    # 生成獨立 ek 個
    ek1 = skewnorm.rvs(1, size = cc1)
    ek2 = skewnorm.rvs(1, size = cc2)
    
    # 計算 q_k
    # qk = (xk-bk*ek)/ak
    qk1 = (xk1-bk1*ek1)/ak1
    qk2 = (xk2-bk2*ek2)/ak2
#     QK[i,] = np.hstack((qk1,qk2))
    
    # 排序q_k大小
    w1 = sorted(qk1)
    w2 = sorted(qk2)
    
    # 填入對應 Answer(Y1, Y2)的機率

    for l in range(len(Ans)):
        Y1 = Ans['kx'][l]
        Y2 = Ans['ky'][l]
# 情況一       
        if Y1 == 0 and Y2 == 0:
            q_min = np.min((w1[0], w2[0]))
            pY[l, i] = skewnorm.cdf(q_min,1)
            
        elif Y1 > 0 and Y1 < cc1 and Y2 == 0:
            if w1[Y1-1] < w2[0]:
                q_min = np.min((w1[Y1], w2[0]))
                q_max = w1[Y1-1]
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)
            else:
                pY[l, i] = 0
                
        elif Y1 == cc1 and Y2 == 0:
            if w1[-1] < w2[0]:
                q_min = w2[0]
                q_max = w1[-1]
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)
            else:
                pY[l, i] = 0
# 情況二               
        elif Y1 == 0 and Y2 > 0 and Y2 < cc2:
            if w1[0] > w2[Y2-1]:
                q_min = np.min((w1[0], w2[Y2]))
                q_max = w2[Y2-1]
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)
            else:
                pY[l, i] = 0
                
        elif Y1 > 0 and Y1 < cc1 and Y2 > 0 and Y2 < cc2:
            if w1[Y1-1] < w2[Y2] and w1[Y1] > w2[Y2-1]:
                q_min = np.min((w1[Y1], w2[Y2]))
                q_max = np.max((w1[Y1-1], w2[Y2-1]))
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)
            else:
                pY[l, i] = 0
                
        elif Y1 == cc1 and Y2 > 0 and Y2 < cc2:
            if w1[-1] < w2[Y2]:
                q_min = w2[Y2]
                q_max = np.max((w1[-1], w2[Y2-1]))
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)  
            else:
                pY[l, i] = 0
# 情況三
        elif Y1 == 0 and Y2 == cc2:
            if w1[0] > w2[-1]:
                q_min = w1[0]
                q_max = w2[-1]
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)
            else:
                pY[l, i] = 0
                
        elif Y1 > 0 and Y1 < cc1 and Y2 == cc2:
            if w1[Y1] > w2[-1]:
                q_min = w1[Y1]
                q_max = np.max((w1[Y1-1], w2[-1]))
                pY[l, i] = skewnorm.cdf(q_min,1) - skewnorm.cdf(q_max,1)
            else:
                pY[l, i] = 0
                
        elif Y1 == cc1 and Y2 == cc2:
            q_max = np.max((w1[-1], w2[-1]))
            pY[l, i] = 1 - skewnorm.cdf(q_max,1)
    print(f'第 {i+1} 次模擬')    
end_total = time.time()
spend = end_total - start_total
print(f'共花費 {spend:.2f} 秒')

第 1 次模擬
第 2 次模擬
第 3 次模擬
第 4 次模擬
第 5 次模擬
第 6 次模擬
第 7 次模擬
第 8 次模擬
第 9 次模擬
第 10 次模擬
第 11 次模擬
第 12 次模擬
第 13 次模擬
第 14 次模擬
第 15 次模擬
第 16 次模擬
第 17 次模擬
第 18 次模擬
第 19 次模擬
第 20 次模擬
第 21 次模擬
第 22 次模擬
第 23 次模擬
第 24 次模擬
第 25 次模擬
第 26 次模擬
第 27 次模擬
第 28 次模擬
第 29 次模擬
第 30 次模擬
第 31 次模擬
第 32 次模擬
第 33 次模擬
第 34 次模擬
第 35 次模擬
第 36 次模擬
第 37 次模擬
第 38 次模擬
第 39 次模擬
第 40 次模擬
第 41 次模擬
第 42 次模擬
第 43 次模擬
第 44 次模擬
第 45 次模擬
第 46 次模擬
第 47 次模擬
第 48 次模擬
第 49 次模擬
第 50 次模擬
第 51 次模擬
第 52 次模擬
第 53 次模擬
第 54 次模擬
第 55 次模擬
第 56 次模擬
第 57 次模擬
第 58 次模擬
第 59 次模擬
第 60 次模擬
第 61 次模擬
第 62 次模擬
第 63 次模擬
第 64 次模擬
第 65 次模擬
第 66 次模擬
第 67 次模擬
第 68 次模擬
第 69 次模擬
第 70 次模擬
第 71 次模擬
第 72 次模擬
第 73 次模擬
第 74 次模擬
第 75 次模擬
第 76 次模擬
第 77 次模擬
第 78 次模擬
第 79 次模擬
第 80 次模擬
第 81 次模擬
第 82 次模擬
第 83 次模擬
第 84 次模擬
第 85 次模擬
第 86 次模擬
第 87 次模擬
第 88 次模擬
第 89 次模擬
第 90 次模擬
第 91 次模擬
第 92 次模擬
第 93 次模擬
第 94 次模擬
第 95 次模擬
第 96 次模擬
第 97 次模擬
第 98 次模擬
第 99 次模擬
第 100 次模擬
共花費 439.88 秒


In [16]:
# 計算一千次模擬 各種組合的估計值平均
Ans_prob = np.zeros(len(Ans))
for i in range(len(Ans)):
    Ans_prob[i] = np.mean(pY[i, :])
# pd.DataFrame(Ans_prob)
prob_com = pd.concat((Ans, pd.DataFrame(Ans_prob)),axis=1)
prob_com.columns=['kx', 'ky', 'prob']

In [17]:
# 計算違約損失金額，並填入風險值
Cost = 2*Ans['kx'] + 1*Ans['ky']
Cost = pd.DataFrame(Cost)
Cost.columns = ['損失金額']
prob_com = pd.concat((Ans, pd.DataFrame(Ans_prob), Cost),axis=1)
prob_com.columns = ['第一群', '第二群', '估計值', '損失金額']

In [18]:
# 計算各風險值估計值平均
X = np.arange(10, 1200, 10)
prob = np.zeros(len(X))
for i in range(len(X)):
    prob[i] = np.sum(prob_com['估計值'][prob_com['損失金額']>X[i]])

# 將違約組合、損失金額、各次模擬機率併成同一表格
prob_temp = pd.concat((Ans, Cost, pd.DataFrame(pY)),axis=1)  

# 每一次模擬，根據其對應的風險值計算違約機率
Risk_prob = np.zeros((len(X), N))
X = np.arange(10, 1200, 10)
for i in range(len(X)):
    num = prob_temp['損失金額']>X[i]
    for j in range(N):
        Risk_prob[i, j] = np.sum(prob_temp[j][num])

In [19]:
# 根據每一次對應風險值的估計值計算總變異數
var = np.zeros(len(X))
for i in range(len(X)):
    var[i] = np.sum((Risk_prob[i, :] - prob[i])**2)/(N-1)

In [20]:
# 將結果儲存為 CSV
DATA = pd.concat([pd.DataFrame(X), pd.DataFrame(prob), pd.DataFrame(var)], axis = 1)
DATA.columns = ['風險值', '估計值', '變異數']
DATA.to_csv('MIS_DEP_CSN_1000_B1_0609.csv', index = False, encoding = 'utf_8_sig')
DATA

,風險值,估計值,變異數
0,10,9.066242e-01,1.862119e-03
1,20,7.861122e-01,2.881770e-03
2,30,6.745223e-01,3.308680e-03
3,40,5.825220e-01,3.622930e-03
4,50,5.000299e-01,2.961853e-03
5,60,4.360030e-01,2.640278e-03
6,70,3.805117e-01,2.075966e-03
7,80,3.296708e-01,1.432446e-03
8,90,2.899580e-01,1.189906e-03
9,100,2.549757e-01,8.553145e-04


In [26]:
DATA.iloc[60][2]

4.0783527616052047e-07